In [3]:
import os
import mne
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from mne.time_frequency import psd_array_multitaper

In [14]:
# Path to the dataset root
data_dir = './ds003523'

# Initialize lists for features and labels
all_features = []
all_labels = []

# Define frequency bands
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 40)
}


# Load participants.tsv to get the Group labels
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
# participa
# # Map Group levels to numeric labels
# group_mapping = {'zero': 0, 'one': 1}  # 0: mTBI, 1: Control
# participants['Group'] = participants['Group'].map(group_mapping)


In [13]:
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
participants

,participant_id,Original_ID,URSI,sex,age,Group
0,sub-001,3044,M87102477,1,23,0
1,sub-002,3009,M87104171,0,30,1
2,sub-003,3042,M87104582,0,19,0
3,sub-004,3074,M87106278,0,19,0
4,sub-005,3076,M87107075,0,22,0
...,...,...,...,...,...,...
86,sub-087,3084,M87192041,0,34,0
87,sub-088,3063,M87192146,1,30,1
88,sub-089,3082,M87196019,0,24,0
89,sub-090,3007,M87196158,1,26,1


### Helper Functions

In [5]:
# Function to compute additional features
def compute_additional_features(epoch_data):
    """
    Computes additional features like:
    - Mean, variance, skewness, and kurtosis for time-domain data
    - Line length (signal complexity)
    """
    from scipy.stats import skew, kurtosis

    # Mean and variance across time points
    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)

    # Skewness and kurtosis across time points
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)

    # Line length (sum of absolute differences between consecutive samples)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

In [16]:
# X = np.vstack(all_features)  # Features for all subjects and sessions
# y = np.hstack(all_labels)  

In [29]:

# Function to compute additional features (mean, variance, skewness, kurtosis, line length)
def compute_additional_features(epoch_data):
    if len(epoch_data.shape) != 2:
        raise ValueError(f"Expected 2D array for epoch_data, got {epoch_data.shape}")

    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

# Initialize variables
all_features = []
all_labels = []
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 40)
}
expected_feature_count = 10  # Total number of features (5 frequency + 5 time-domain)

# Loop through each subject
for subject in range(1, 2):  # Adjust range as needed
    subject_id = f"sub-{subject:03d}"  # Format: sub-001, sub-002, etc.
    subject_id_lower = subject_id.lower()

    # Check if subject exists in participants.tsv
    if subject_id_lower not in participants['participant_id'].values:
        print(f"Subject {subject_id} not found in participants.tsv. Skipping.")
        continue

    # Locate the subject directory
    subject_dir = os.path.join(data_dir, subject_id)
    if not os.path.exists(subject_dir):
        print(f"Subject directory {subject_id} not found. Skipping.")
        continue

    # Find all available sessions for the subject
    sessions = [s for s in os.listdir(subject_dir) if s.startswith('ses-')]

    for session in sessions:
        session_path = os.path.join(subject_dir, session, 'eeg')

        # Retrieve session data using DataLad
        os.system(f'datalad get {session_path}')

        # Check if the .set file exists for this session
        eeg_file = os.path.join(session_path, f"{subject_id}_{session}_task-VisualWorkingMemory_eeg.set")
        if not os.path.exists(eeg_file):
            print(f"Data for {subject_id}, {session} not found. Skipping.")
            continue

        # Load the EEG data
        raw = mne.io.read_raw_eeglab(eeg_file, preload=True)

        # Preprocess the data
        raw.filter(1., 40., fir_design='firwin')

        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Define epochs
        tmin, tmax = -0.2, 0.5
        epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)

        # Compute PSD for the entire frequency range
        psd = epochs.compute_psd(method='multitaper', fmin=0.5, fmax=40)
        psd_data = psd.get_data()  # Shape: (n_epochs, n_channels, n_freqs)
        freqs = psd.freqs

        # Compute features for each frequency band
        band_powers = {}
        for band, (fmin, fmax) in band_ranges.items():
            band_indices = (freqs >= fmin) & (freqs < fmax)
            band_power = psd_data[:, :, band_indices].mean(axis=2)  # Mean PSD within the band
            band_powers[band] = band_power.mean(axis=1)  # Aggregate across channels

        # Extract time-domain data from epochs
        epoch_data = epochs.get_data().mean(axis=1)  # Aggregate across channels (shape: n_epochs, n_times)

        # Compute additional features
        mean_values, var_values, skew_values, kurtosis_values, line_length = compute_additional_features(epoch_data)

        # Combine all features into a single array
        try:
            session_features = np.column_stack([
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], mean_values,
                var_values, skew_values, kurtosis_values, line_length
            ])
            print(f"Session features shape for {subject_id}, {session}: {session_features.shape}")
        except ValueError as e:
            print(f"Feature dimension mismatch for {subject_id}, {session}: {e}")
            continue

        # Pad or truncate to maintain consistent dimensions
        if session_features.shape[1] != expected_feature_count:
            print(f"Inconsistent feature count. Padding/truncating for {subject_id}, {session}")
            session_features = np.pad(
                session_features,
                ((0, 0), (0, expected_feature_count - session_features.shape[1])),
                mode='constant'
            )

        all_features.append(session_features)

        # Get the group label for the subject
        group_label = participants.loc[participants['participant_id'] == subject_id_lower, 'Group'].values
        if len(group_label) == 0 or pd.isna(group_label[0]):
            print(f"No valid group label found for {subject_id}. Skipping.")
            continue

        # Repeat the label for each epoch in the session
        all_labels.append(np.repeat(group_label[0], session_features.shape[0]))

Reading c:\Users\Rahul\Documents\FREELANCING\1.AppliedSkil\isha_project\ds003523\sub-001\ses-01\eeg\sub-001_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 697174  =      0.000 ...  1394.348 secs...
Filtering raw data in 1 contiguous segment


C:\Users\Rahul\AppData\Local\Temp\ipykernel_3064\3813580270.py:53: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True)


Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s


Used Annotations descriptions: [np.str_('S  1'), np.str_('S  2'), np.str_('S  3'), np.str_('S 50'), np.str_('S 51'), np.str_('S 52'), np.str_('S100'), np.str_('S101'), np.str_('S200'), np.str_('S201'), np.str_('boundary')]
Not setting metadata
511 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 511 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
delta power shape: (510,)
theta power shape: (510,)
alpha power shape: (510,)
beta power shape: (510,)
gamma power shape: (510,)


In [33]:

# Combine all features and labels across sessions
if not all_features or not all_labels:
    print("No features or labels collected. Exiting.")
    raise ValueError("No features or labels collected.")

# Combine features and labels
X = np.vstack(all_features)  # Features for all subjects and sessions
y = np.hstack(all_labels)    # Labels for all subjects and sessions

# Save the training data
feature_columns = [
    'delta', 'theta', 'alpha', 'beta', 'gamma',
    'mean', 'variance', 'skewness', 'kurtosis', 'line_length'
]
training_data = pd.DataFrame(X, columns=feature_columns)
training_data['label'] = y
training_data.to_csv('eeg_training_data_with_features.csv', index=False)
print("Training data saved to 'eeg_training_data_with_features.csv'")

# Feature and label inspection
print(f"Feature shape: {X.shape}")
print(f"Label distribution:\n{pd.Series(y).value_counts()}")


Reading c:\Users\Rahul\Documents\FREELANCING\1.AppliedSkil\isha_project\ds003523\sub-001\ses-01\eeg\sub-001_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 697174  =      0.000 ...  1394.348 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)



C:\Users\Rahul\AppData\Local\Temp\ipykernel_3064\104574647.py:64: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.4s


Used Annotations descriptions: [np.str_('S  1'), np.str_('S  2'), np.str_('S  3'), np.str_('S 50'), np.str_('S 51'), np.str_('S 52'), np.str_('S100'), np.str_('S101'), np.str_('S200'), np.str_('S201'), np.str_('boundary')]
Not setting metadata
511 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 511 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-001, ses-01: (510, 10)
Training data saved to 'eeg_training_data_with_features.csv'
Feature shape: (510, 10)
Label distribution:
0    510
Name: count, dtype: int64


In [35]:
training_data.head(21)

,delta,theta,alpha,beta,gamma,mean,variance,skewness,kurtosis,line_length,label
0,0.000121,0.000066,1.041631e-05,4.093264e-07,1.703079e-07,-2.759644e-07,2.030720e-08,-1.782830,2.544229,0.002042,0
1,0.000223,0.000091,9.860508e-07,2.634769e-07,1.319152e-07,1.014448e-04,4.067719e-08,0.469868,-1.111458,0.001605,0
2,0.000521,0.000247,2.919830e-05,3.682986e-06,4.751889e-07,5.155409e-05,1.059130e-07,0.011857,-0.114614,0.004012,0
3,0.000054,0.000024,1.905838e-06,3.589238e-07,1.915836e-07,-6.625469e-05,7.938603e-09,0.043146,-1.428369,0.001619,0
4,0.000098,0.000046,6.124056e-06,2.865901e-06,4.610708e-07,1.745370e-04,1.916182e-08,-0.702796,0.526098,0.002474,0
5,0.000188,0.000107,2.140490e-05,2.947721e-06,3.079022e-07,-1.509964e-04,3.715844e-08,-1.224003,0.505169,0.003225,0
6,0.000273,0.000118,8.566411e-06,5.676609e-07,1.753410e-07,-2.532423e-04,6.251097e-08,-0.392299,-1.353086,0.002111,0
7,0.000068,0.000030,5.305230e-06,2.460849e-06,5.262691e-07,-7.961382e-05,1.352736e-08,-0.375771,-0.654692,0.002634,0
8,0.000367,0.000181,1.472465e-05,2.473436e-07,1.762354e-07,-1.488635e-04,5.504656e-08,-1.434611,0.755591,0.002432,0
9,0.000117,0.000060,1.205118e-05,2.832324e-06,2.915472e-07,-9.359116e-05,2.035195e-08,-1.397455,0.720088,0.002831,0
